In [68]:
import urllib3
import requests
import os
import zipfile
import csv

import swat

In [69]:
from swat import *
cassession = CAS('sasserver.demo.sas.com', 5570, authinfo='~/.authinfo', caslib="casuser")
cassession.loadactionset(actionset="table")

NOTE: Added action set 'table'.


[actionset]

 'table'

+ Elapsed: 0.00174s, user: 0.002s, mem: 0.0553mb

In [100]:
url = 'http://nwis.waterdata.usgs.gov/nwis/uv?cb_00060=on&cb_00065=on&format=rdb&site_no=02175000&period=&begin_date=2016-06-01&end_date=2016-10-19'
http = urllib3.PoolManager()
urllib3.disable_warnings()
response = http.request('GET',url)
with open('temp.dat','wb') as f:
    f.write(response.data)
f.close()

In [101]:
#Clean the headers out of the extract. This is very regular in USGS streamflow through 2017
f2 = open("temp2.dat","w")
for line in open("temp.dat","r").readlines():
    if line[0] != "#" and line[0] != "5":
        f2.write(line)
f2.close()

In [96]:
in_txt = csv.reader(open("temp2.dat", "r"), delimiter = '\t')
out_csv = csv.writer(open("temp.csv", 'w'))

out_csv.writerows(in_txt)

In [95]:
#cassession.dropTable("TEMP")

NOTE: Cloud Analytic Services dropped table TEMP from caslib CASUSER(viyauser).


+ Elapsed: 0.000411s, user: 0.001s, mem: 0.0302mb

In [97]:
cas_table = cassession.upload("temp.csv").casTable

NOTE: Cloud Analytic Services made the uploaded file available as table TEMP in caslib CASUSER(viyauser).
NOTE: The table TEMP has been created in caslib CASUSER(viyauser) from binary data uploaded to Cloud Analytic Services.


In [98]:
cas_table.head()

,agency_cd,site_no,datetime,tz_cd,128196_00065,128196_00065_cd,128198_00060,128198_00060_cd
0,USGS,2175000.0,2016-06-01 00:00,EST,5.61,P,1860.0,P
1,USGS,2175000.0,2016-06-01 01:00,EST,5.61,P,1860.0,P
2,USGS,2175000.0,2016-06-01 02:00,EST,5.61,P,1860.0,P
3,USGS,2175000.0,2016-06-01 03:00,EST,5.60,P,1850.0,P
4,USGS,2175000.0,2016-06-01 04:00,EST,5.60,P,1850.0,P


## Identify High, Medium and Low streamflow clusters

In [104]:
cassession.loadactionset(actionset='clustering')
cassession.kclus(table={"name":"TEMP"}, inputs={"128198_00060"},nClusters=3,seed=123,maxIters=100,init="RAND")

NOTE: Added action set 'clustering'.
NOTE:  Using SEED=123.
NOTE: Running for input data for K 3.
NOTE: Clustering is finished. STOPVALUE is satisfied for STOPCRITERION=CLUSTER_CHANGE.


,Type,N
0,Number of Observations Read,3399.0
1,Number of Observations Used,3399.0
,Parameter,Setting
0,Maximum Iterations,100
1,Stop Criterion,Cluster Change
2,Stop Criterion Value,0
3,Clusters,3
4,Initialization,Rand
5,Seed,123
6,Distance,Euclidean


# Close the session

In [5]:
cassession.close()